In [2]:
import torch
import lovely_tensors

lovely_tensors.monkey_patch()

In [2]:
p = "../runs/ppo/final_mixed_objective_training_666_2025-05-16_19-08-25/weights/policy_final.pth"

In [3]:
sd1 = torch.load(p, map_location="cpu")
sd1

OrderedDict([('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.0.weight',
              tensor[64, 15] n=960 (3.8Kb) x∈[-0.530, 0.592] μ=0.005 σ=0.178),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.0.bias',
              tensor[64] x∈[-0.152, 0.224] μ=-0.000 σ=0.072),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.1.weight',
              tensor[64] x∈[0.879, 1.056] μ=0.978 σ=0.035),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.1.bias',
              tensor[64] x∈[-0.108, 0.129] μ=-0.004 σ=0.045),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.3.weight',
              tensor[64, 64] n=4096 (16Kb) x∈[-0.655, 0.593] μ=-0.001 σ=0.178),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.3.bias',
              tensor[64] x∈[-0.078, 0.092] μ=0.001 σ=0.038),
             ('module.0.module.0.modul

In [4]:
for k, v in sd1.items():
    if v.shape[-1] == 128:
        print(k, v)

module.0.module.0.module.0.module.encoders.Heightmap.encoder.8.weight tensor[64, 128] n=8192 (32Kb) x∈[-0.492, 0.516] μ=-0.002 σ=0.129
module.0.module.0.module.1.module.0.mlp.0.weight tensor[64, 128] n=8192 (32Kb) x∈[-0.512, 0.457] μ=-0.000 σ=0.131
module.1.module.0.module.encoders.Heightmap.encoder.8.weight tensor[64, 128] n=8192 (32Kb) x∈[-1.291, 1.213] μ=-0.005 σ=0.294
module.1.module.1.module.mlp.0.weight tensor[64, 128] n=8192 (32Kb) x∈[-1.125, 1.089] μ=0.001 σ=0.267


In [ ]:
from pathlib import Path

networks_dir = "../modified_networks/transferred_with_preference/"
networks_dir = Path(networks_dir)
networks_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
from copy import deepcopy

sd = deepcopy(sd1)
for k, v in sd.items():
    if v.shape[-1] == 128 and "Heightmap" not in k:
        new_param = torch.zeros(*v.shape[:-1], 129)  # account for the new preference dimension
        new_param[..., :-1] = v
        new_param[..., -1] = 0.5
        sd[k] = new_param
sd

OrderedDict([('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.0.weight',
              tensor[64, 15] n=960 (3.8Kb) x∈[-0.530, 0.592] μ=0.005 σ=0.178),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.0.bias',
              tensor[64] x∈[-0.152, 0.224] μ=-0.000 σ=0.072),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.1.weight',
              tensor[64] x∈[0.879, 1.056] μ=0.978 σ=0.035),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.1.bias',
              tensor[64] x∈[-0.108, 0.129] μ=-0.004 σ=0.045),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.3.weight',
              tensor[64, 64] n=4096 (16Kb) x∈[-0.655, 0.593] μ=-0.001 σ=0.178),
             ('module.0.module.0.module.0.module.encoders.LocalStateVector.mlp.mlp.3.bias',
              tensor[64] x∈[-0.078, 0.092] μ=0.001 σ=0.038),
             ('module.0.module.0.modul

In [7]:
p_vecnom = p.replace("policy", "vecnorm")
torch.save(sd, networks_dir / "policy.pth")
torch.save(torch.load(p_vecnom, map_location="cpu"), networks_dir / "vecnorm.pth")

In [23]:
policies = [
    "final_coarse_gaussian_terrain_thesis_98_2025-05-12_12-07-12",
    "final_mixed_objective_training_666_2025-05-16_19-08-25",
    "mixed_training_smoothness_2025-05-20_06-47-01",
    "final_fine_gaussian_terrain_thesis_98_2025-05-12_13-34-12",
]

from pathlib import Path
from copy import deepcopy

p_root = Path("../runs/ppo/")

p_sds = [torch.load(p_root / f"{policy}/weights/policy_final.pth", map_location="cpu") for policy in policies]

vecnorm_sds = [torch.load(p_root / f"{policy}/weights/vecnorm_final.pth", map_location="cpu") for policy in policies]

In [24]:
sd_avg = deepcopy(p_sds[0])
vecnorm_avg = deepcopy(vecnorm_sds[0])

for k, v in sd_avg.items():
    for i in range(1, len(p_sds)):
        sd_avg[k] += p_sds[i][k]
    sd_avg[k] /= len(p_sds)

for k, v in vecnorm_avg["_extra_state"].items():
    for i in range(1, len(vecnorm_sds)):
        vecnorm_avg["_extra_state"][k] += vecnorm_sds[i]["_extra_state"][k]
    vecnorm_avg["_extra_state"][k] /= len(vecnorm_sds)

In [25]:
vecnorm_avg, vecnorm_sds[0]

(OrderedDict([('_extra_state',
               {'LocalStateVector_sum': tensor[15] x∈[-356.790, 1.770e+04] μ=5.229e+03 σ=7.822e+03,
                'LocalStateVector_ssq': tensor[15] x∈[8.913, 1.036e+04] μ=3.020e+03 σ=4.334e+03,
                'LocalStateVector_count': tensor[1] [3.840e+04],
                'reward_sum': tensor[1] [-6.960e+03],
                'reward_ssq': tensor[1] [1.773e+04],
                'reward_count': tensor[1] [3.840e+04]})]),
 OrderedDict([('_extra_state',
               {'LocalStateVector_sum': tensor[15] x∈[-492.177, 1.336e+04] μ=3.494e+03 σ=5.413e+03,
                'LocalStateVector_ssq': tensor[15] x∈[11.335, 8.346e+03] μ=2.141e+03 σ=3.179e+03,
                'LocalStateVector_count': tensor[1] [2.560e+04],
                'reward_sum': tensor[1] [5.083e+03],
                'reward_ssq': tensor[1] [1.310e+04],
                'reward_count': tensor[1] [2.560e+04]})]))

In [26]:
networks_dir = "../modified_networks/top_all_averaged/"
networks_dir = Path(networks_dir)
networks_dir.mkdir(parents=True, exist_ok=True)

torch.save(sd_avg, networks_dir / "policy.pth")
torch.save(vecnorm_avg, networks_dir / "vecnorm.pth")